## using gpt to query

### data cleaning

In [4]:
## unclear if i even need this 
import pycountry
import difflib

def clean_country_name(name):
    """Find the best guess for a country name."""
    countries = [country.name for country in pycountry.countries]
    closest_match = difflib.get_close_matches(name, countries, n=1)
    return closest_match[0] if closest_match else None

# Example usage
clean_country_name("PRC")


In [17]:
%pip install googlesearch-python
%pip install newspaper3k

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 3.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 7.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 7.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 8.7 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13541 sha256=6858f819db2adfb72a76064e2cc524d4345e96ec949dfb7c6950726d0ac9cfc8
  Stored in directory: /Users/kaylahuang/Library/Caches/pip/wheels/a5/91/9f/00d66475960891a64867914273fcaf

In [5]:
from googlesearch import search

def google_search_links(query, num_results=10):
    """Return the top N links from a Google search."""
    return [link for i, link in enumerate(search(query, num_results))]

# query = "why did the conflict in south sudan happen"
query = "timeline for the conflict in south sudan"
links = google_search_links(query, 5)
print(links)


['https://concernusa.org/news/timeline-south-sudan-history/', 'https://www.bbc.com/news/world-africa-14019202', 'https://www.aljazeera.com/news/2020/2/22/timeline-south-sudan-since-independence', 'https://www.aljazeera.com/news/2023/5/31/fighting-in-sudan-a-timeline-of-key-events', 'https://africacenter.org/wp-content/uploads/2019/12/Timeline-of-South-Sudan-Peace-Agreements-and-Violence-printable.pdf', 'https://www.rescuesouthsudan.org/sudan-timeline-of-events/', 'https://www.cfr.org/global-conflict-tracker/conflict/civil-war-south-sudan']


In [3]:
# import requests
# from bs4 import BeautifulSoup

# def bs_parser(url):
#     """Return the text from a webpage."""
#     r = requests.get(url)
#     soup = BeautifulSoup(r.text, 'html.parser')
#     for script_or_style in soup(['script', 'style']):
#         script_or_style.extract()
    
#     text = soup.get_text()
#     body = soup.find('body')
#     if body:
#         text = body.get_text()
    
#     text = ' '.join(text.split())
#     return text

# for url in links:
#     print(bs_parser(url))

In [7]:
from newspaper import Article

# Example URL
url = 'https://concernusa.org/news/timeline-south-sudan-history/'

def read_article(url):
    article = Article(url)
    article.download()
    article.parse()
    return article 
    # print("Title:", article.title)
    # print("Text:", article.text)
    # print("Publish Date:", article.publish_date)
    # article.nlp()
    # print("Keywords:", article.keywords)
    # print("Summary:", article.summary)

with open('south_sudan.txt', 'w') as f:
    for url in links:
        article = read_article(url)
        f.write(article.title)
        f.write(article.text)
        f.write('\n\n-----------------------------------\n\n')

with open('south_sudan.txt', 'r') as f:
    text_for_gpt = f.read()


## the actual query

In [8]:
import os
import openai
from openai import OpenAI
openai.organization = "org-raWgaVqCbuR9YlP1CIjclYHk" # Harvard
openai.api_key = os.getenv("OPENAI_API_KEY")

print(True if openai.api_key else False)

True


In [9]:
client = OpenAI()

with open('res.txt', 'w') as txtfile:
    completion = client.chat.completions.create(
        # turbo (1106-preview) has 128k context window, about 300 pages of text
        model="gpt-4-1106-preview", # test with: gpt-3.5-turbo, run final: gpt-4-1106-preview
        messages=[
            {"role": "system", "content": 
                """You are a conflict analyst. You are writing a report on the conflict in South Sudan. Someone has provided you with a few articles which outline the timeline of the conflict. You want to summarize the timeline in your report."""},
            {"role": "user", "content": f"Here are the bodies of the articles: {text_for_gpt}"}
        ]
    )

    gpt_summary = completion.choices[0].message.content
    
    txtfile.write(f"{gpt_summary}\n")